# Seismic Event Detection using SeisBench

This notebook demonstrates the complete workflow for seismic event detection using SeisBench models,
following the approach from real-time detection systems.

## Workflow
1. Load pre-trained SeisBench model (PhaseNet, EQTransformer, or QuakeXNet)
2. Download seismic data from FDSN client
3. Run model inference to get probability predictions
4. Detect event windows from probabilities
5. Interactive visualization for quality control
6. Export results

In [ ]:
# Import required libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from src.seisbench_models import create_detector, SeismicDetector
from src.detect import smooth_moving_avg, detect_event_windows, multi_class_detection
from src.interactive_plots import plot_detection_results, interactive_detection_viewer, plot_event_summary
from src.utils import ensure_dir

# For interactive plots in Jupyter
%matplotlib widget
# Or use: %matplotlib inline

print("✓ Imports successful!")

## 1. Configuration and Setup

In [ ]:
# Configuration
config = {
    # Model settings
    'model_name': 'phasenet',  # Options: 'phasenet', 'eqtransformer', 'gpd'
    'model_version': 'original',
    'use_quakescope': False,  # Set to True to use QuakeScope models
    'device': 'auto',  # 'auto', 'cpu', or 'cuda'
    
    # Data settings
    'network': 'UW',
    'station': 'RATT',
    'channel': 'HH*',  # HH* for high-bandwidth, BH* for broadband
    'location': '*',
    
    # Time window (use recent data or specify custom)
    'use_recent': True,  # If True, use last 24 hours
    'hours_back': 24,
    'starttime': '2024-01-01T00:00:00',  # Used if use_recent=False
    'endtime': '2024-01-01T01:00:00',
    
    # Detection parameters
    'threshold': 0.5,
    'min_duration': 10,  # samples
    'merge_distance': 50,  # samples
    'apply_smoothing': True,
    'smooth_window': 100,
    
    # Output settings
    'save_plots': True,
    'save_results': True,
}

# Create output directories
ensure_dir('../plots')
ensure_dir('../logs')

print("Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

## 2. Load Pre-trained SeisBench Model

We'll load a pre-trained model from SeisBench. These models are trained on large
seismic datasets and can detect various types of events.

In [ ]:
# Create detector
print(f"Loading {config['model_name']} model...")

detector = create_detector(
    model_name=config['model_name'],
    version=config['model_version'],
    device=config['device'],
    use_quakescope=config['use_quakescope']
)

print(f"✓ Model loaded successfully on {detector.device}")
print(f"  Model: {detector.model_name}")
print(f"  Version: {detector.version}")

## 3. Download Seismic Data

Download data from IRIS/FDSN data center.

In [ ]:
# Set up time window
if config['use_recent']:
    endtime = UTCDateTime.now()
    starttime = endtime - config['hours_back'] * 3600
else:
    starttime = UTCDateTime(config['starttime'])
    endtime = UTCDateTime(config['endtime'])

print(f"Time window:")
print(f"  Start: {starttime}")
print(f"  End: {endtime}")
print(f"  Duration: {(endtime - starttime) / 3600:.1f} hours")

# Download data
print(f"\nDownloading data for {config['network']}.{config['station']}...")

client = Client("IRIS")

try:
    stream = client.get_waveforms(
        network=config['network'],
        station=config['station'],
        channel=config['channel'],
        location=config['location'],
        starttime=starttime,
        endtime=endtime
    )
    
    print(f"✓ Downloaded {len(stream)} traces")
    print(f"\nStream info:")
    print(stream)
    
except Exception as e:
    print(f"✗ Error downloading data: {e}")
    print("\nTip: Try different station/network or time window")
    raise

## 4. Run Model Inference

Use SeisBench's `annotate()` method to get probability predictions.

In [ ]:
print("Running model inference...")

# Run inference using SeisBench annotate
annotated_stream = detector.annotate(
    stream,
    batch_size=32,
    overlap=0
)

print(f"✓ Inference complete")
print(f"\nAnnotated stream:")
print(annotated_stream)

# Extract predictions by class
# Common class names: P (P-wave), S (S-wave), N (Noise)
# or: eq (earthquake), px (explosion), su (surface event)
probabilities = detector._extract_predictions(annotated_stream)

print(f"\nPrediction classes:")
for class_name, probs in probabilities.items():
    print(f"  {class_name}: {len(probs)} samples, max prob: {np.max(probs):.3f}")

## 5. Detect Event Windows

Apply smoothing and threshold-based detection to find event windows.

In [ ]:
print("Detecting event windows...")

# Detect events for each class
events = multi_class_detection(
    probabilities,
    threshold=config['threshold'],
    min_duration=config['min_duration'],
    merge_distance=config['merge_distance'],
    apply_smoothing=config['apply_smoothing'],
    smooth_window=config['smooth_window']
)

print(f"✓ Detection complete")
print(f"\nDetected events:")
total_events = 0
for class_name, class_events in events.items():
    print(f"  {class_name}: {len(class_events)} events")
    total_events += len(class_events)

print(f"\nTotal events: {total_events}")

## 6. Create Event Summary DataFrame

In [ ]:
# Convert events to DataFrame for easier analysis
event_records = []
sampling_rate = stream[0].stats.sampling_rate

for class_name, class_events in events.items():
    for event in class_events:
        start_idx = event['start']
        end_idx = event['end']
        start_time = starttime + (start_idx / sampling_rate)
        end_time = starttime + (end_idx / sampling_rate)
        
        event_records.append({
            'station': config['station'],
            'network': config['network'],
            'class': class_name,
            'start_index': start_idx,
            'end_index': end_idx,
            'duration': event['duration'],
            'start_time': str(start_time),
            'end_time': str(end_time),
            'max_prob': event['max_prob'],
            'mean_prob': event['mean_prob'],
            'auc': event['area_under_curve'],
        })

if event_records:
    df_events = pd.DataFrame(event_records)
    print("Event Summary:")
    print(df_events.to_string())
    
    # Save to CSV
    if config['save_results']:
        csv_path = f"../logs/{config['station']}_{starttime.date}_events.csv"
        df_events.to_csv(csv_path, index=False)
        print(f"\n✓ Results saved to {csv_path}")
else:
    print("No events detected")
    df_events = None

## 7. Visualization - Detection Results

Plot waveforms with probability predictions and detected event windows.

In [ ]:
# Plot detection results
save_path = None
if config['save_plots']:
    save_path = f"../plots/{config['station']}_{starttime.date}_detection.png"

fig, axes = plot_detection_results(
    stream,
    probabilities,
    events,
    sampling_rate=sampling_rate,
    figsize=(15, 10),
    save_path=save_path
)

plt.show()

## 8. Interactive Quality Control Viewer

Use interactive plots to review detections and toggle event classes.

In [ ]:
# Interactive viewer with controls
if total_events > 0:
    fig_interactive = interactive_detection_viewer(
        stream,
        probabilities,
        events,
        sampling_rate=sampling_rate
    )
else:
    print("No events to display in interactive viewer")

## 9. Event Statistics Summary

In [ ]:
# Plot event statistics
if total_events > 0:
    save_path = None
    if config['save_plots']:
        save_path = f"../plots/{config['station']}_{starttime.date}_summary.png"
    
    fig_summary, axes_summary = plot_event_summary(
        events,
        sampling_rate=sampling_rate,
        figsize=(12, 8),
        save_path=save_path
    )
    
    plt.show()
else:
    print("No events to summarize")

## 10. Export Results for Further Analysis

In [ ]:
# Summary statistics
if df_events is not None:
    print("\n" + "="*60)
    print("DETECTION SUMMARY")
    print("="*60)
    print(f"Station: {config['network']}.{config['station']}")
    print(f"Time window: {starttime} to {endtime}")
    print(f"Duration: {(endtime - starttime) / 3600:.1f} hours")
    print(f"\nTotal events detected: {len(df_events)}")
    print("\nEvents by class:")
    print(df_events['class'].value_counts())
    print("\nProbability statistics:")
    print(df_events[['class', 'max_prob', 'mean_prob']].groupby('class').describe())
    print("="*60)
else:
    print("No events detected in this time window")

## Next Steps

- **Try different models**: PhaseNet, EQTransformer, GPD
- **Adjust detection parameters**: threshold, min_duration, merge_distance
- **Process multiple stations**: Loop through a station list
- **Real-time monitoring**: Adapt for continuous detection
- **Custom models**: Load QuakeScope or custom trained weights
- **Advanced filtering**: Apply additional quality control filters
- **Catalog comparison**: Compare with earthquake catalogs

## References

- SeisBench Documentation: https://seisbench.readthedocs.io/
- ObsPy Documentation: https://docs.obspy.org/
- QuakeScope Models: https://github.com/quakescope
- PNW Seismic Event Classification: https://github.com/Akashkharita/PNW_Seismic_Event_Classification